In [1]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import os
from gensim import corpora, models, similarities
import joblib
import pickle
import numpy as np
import jieba.analyse
import jieba
import codecs
import pandas as pd
import re

# Load lyrics data

In [2]:
df = pd.read_csv ('./data/lyric_df.csv')
df

,Singer,Name,Lyric,Category,seg_text,remove_synonym,top_10
0,A-Lin,One Life,白雲撞進藍天 激起昨天畫面我想起你的臉好久不見 在遠方的My Friend不論晴天雨天 也會...,勵志友情,"白雲撞, 進藍天, 激起, 昨天, 畫面, 想起, 臉, 好久, 不見, 遠方, MyFri...",白雲撞 進藍天 激起 昨天 畫面 想起 臉 好久 不見 遠處 MyFriend 晴天 雨天 ...,"['Life', '我們', '一天', '期待', '哪邊', '邊要', '出現', '..."
1,A-Lin,最好的朋友在身邊,一起笑一起哭一起鬧久別的回憶又浮現一點點一天天一年年最好的朋友在身邊一起走一起跑一起跳相聚到...,勵志友情,"一起, 笑, 一起, 哭, 一起, 鬧久別, 回憶, 浮現, 一點點, 一天天, 一年, 年...",一起 笑 一起 哭 一起 鬧久別 回憶 浮現 一些 一天天 一年 年 最好 朋友 身邊 一起...,"['一起', '一天天', '永在', '鬧久別', '回憶', '浮現', '一點點', ..."
2,A-Lin,愛的可能,你出現我身邊 像個奇蹟發生沒想到會是你 讓我如此失魂我心中的感覺是這樣陌生快樂的牽掛在相聚的...,勵志友情,"出現, 身邊, 蹟, 發生, 想到, 失魂, 心中, 感覺, 陌生, 快樂, 牽掛, 相聚,...",出現 身邊 蹟 發生 想 失魂 心中 感覺 陌生 快樂 牽掛 相見 每一分 曾以 見過 所有...,"['我會', '不會', '記得', '孤單', '時候', '因為', '還有', '我的..."
3,A-Lin,太太太耐斯,孩子還沒睡 拜託別鬧手遊聲太吵 狗狗在叫碗盤在洗碗槽 主婦的煩惱惱公不瞭 他先洗澡奶瓶把香水...,反抗反駁反諷,"孩子, 還沒, 睡, 拜, 託, 別鬧, 手遊聲, 太吵, 狗狗, 碗, 盤, 洗碗, 槽,...",孩子 還沒 睡 拜 託 別鬧 手遊聲 太吵 狗狗 碗 盤 洗碗 槽 主婦 煩惱 惱公 瞭 先...,"['太太', '耐斯', '什麼', '超過', '太過', '發脾氣', '心軟', '忘..."
4,A-Lin,抱歉 我不抱歉,恨 什麼叫恨 這個單字 好陌生我只記得 一路走來 愛的人就算他們 多可恨等 雖然溫柔 還打不...,反抗反駁反諷,"恨, 恨, 單字, 好, 陌生, 只, 記得, 一路, 走來, 愛的, 人, 可恨, 溫柔,...",恨 恨 單字 好 陌生 只 記得 一路 走過來 愛的 人 可恨 溫柔 還打 不開 一些 門雖...,"['抱歉', '他們', '每枝', '什麼', '我們', '每個', '每張', '太真..."
...,...,...,...,...,...,...,...
2201,陳奕迅,黑暗中漫舞,為何未能學會起舞便已抱緊你誰料到資質不配合你左腳舉起了便要別離為何未能讓我衰老便要放開你陪你...,悲傷離別,"未能, 學會, 起舞, 已, 抱, 緊, 料到, 資質, 配合, 左腳, 舉起, 別離, 未...",未能 學 起舞 已經 抱 緊 料到 資質 配合 左腳 舉起 別離 未能 衰老 放開 陪伴 跳...,"['為何', '未能', '左腳', '美不美', '捱到', '學會', '資質', '舉..."
2202,陳奕迅,傷信,重讀著你的告別信 抑壓了暗湧雖不信寫的話 竟可以這麼重但再哭亦無用 徐徐又當這信是你 緊貼我...,悲傷離別,"重讀, 告別, 信, 抑壓, 暗湧, 雖不信, 寫, 話, 重, 再, 哭, 亦, 無用, ...",重讀 告別 信 抑壓 暗湧 雖不信 寫 話 重 再 哭 亦 無用 徐徐 又當 這信 緊貼 抱...,"['抑壓', '怎麼', '重讀', '告別', '暗湧', '雖不信', '這麼', '無..."
2203,陳奕迅,歲月如歌,愛上了 看見你 如何不懂謙卑去講心中理想 不會俗氣猶如看得見晨曦 才能歡天喜地抱著你 我每次...,悲傷離別,"愛上, 懂謙卑, 去講, 心中, 理想, 俗氣, 猶如, 晨曦, 歡天, 喜地, 抱, 每次...",愛上 懂謙卑 去講 心中 夢想 俗氣 猶如 晨曦 歡天 喜地 抱 每次 回來 驚喜 也許 一...,"['也許', '愛上', '看見', '懂謙卑', '去講', '不會', '俗氣', '猶..."
2204,陳奕迅,遇見了你(國),遇見了你 讓生命美好我看見繁華的春天愛情如陽光閃耀我給你我的愛離開了我 你默默低頭我看見往事...,悲傷離別,"遇見, 生命, 美好, 看見, 繁華, 春天, 愛情, 如陽光, 閃耀, 我給, 愛離開, ...",遇到 生命 美好 看見 繁華 春天 愛情 如陽光 閃耀 我給 愛離開 靜靜 低頭 往事 如雲...,"['離開', '我給', '帶著', '愛帶', '飛到', '邊到', '一個', '傷終..."


# Import testing lyric and preprocess

In [3]:
def remove_punctuation(line):
    stopwords = [line.strip() for line in open('./data/stopwords.txt', 'r', encoding='utf-8').readlines()]
    line = str(line)
    if line.strip() == '':
        return ''
    re_han = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = re_han.sub('', line)
    cut = [w for w in list(jieba.cut(line)) if w not in stopwords]
    res = " ".join(cut)
    return res

In [4]:
test_lyric = "會不會 有一天 時間真的能倒退 退回 你的我的 回不去的 悠悠的歲月 也許會 有一天 世界真的有終點 也要和你舉起回憶釀的甜 和你再乾一杯 如果說 要我選出 代表青春 那個畫面 浮現了 那滴眼淚 那片藍天 那年畢業 那一張 邊哭邊笑 還要擁抱 是你的臉 想起來 可愛可憐 可歌可泣 可是多懷念 懷念總是 突然懷念 不談條件 當回憶 衝破考卷 衝出歲月 在我眼前 我和你 流著汗水 喝著汽水 在操場邊 說好了 無論如何 一起走到 未來的世界 現在就是 那個未來 那個世界 為什麼 你的身邊 我的身邊 不是同一邊 友情曾像 諾亞方舟 堅強誓言 只是我 望著海面 等著永遠 模糊了視線 會不會 有一天 時間真的能倒退 退回 你的我的 回不去的 悠悠的歲月 也許會 有一天 世界真的有終點 也要和你舉起回憶釀的甜 和你再乾一杯 這些年 買了四輪 買了手錶 買了單眼 卻發現 追不到的 停不了的 還是那些 人生是 只有認命 只能宿命 只好宿醉 只剩下 高的笑點 低的哭點 卻沒成熟點 成熟就是 幻想幻滅 一場磨練 為什麼 只有夢想 越磨越小 小到不見 有時候 好想流淚 好想流淚 卻沒眼淚 期待會 你會不會 他會不會 開個同學會 他在等你 你在等我 我在等誰 又是誰 孩子沒睡 電話沒電 心情沒準備 天空不斷 黑了又亮 亮了又黑 那光陰 滄海桑田 遠走高飛 再沒力氣追 終究會 有一天 我們都變成昨天 是你 陪我走過 一生一回 匆匆的人間 有一天 就是今天 今天就是有一天 說出一直沒說 對你的感謝 和你再乾一杯 再乾一杯永遠 喝了就能萬歲 歲歲和年年 時間都停了 他們都回來了 懷念的人啊 等你的來到 時間都停了 他們都回來了 懷念的人啊 等你的來到"
test_lyric_seg = remove_punctuation(test_lyric)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.149 seconds.
Prefix dict has been built successfully.


# Get categories

## Load TfidfVectorizer and transform lyric to vector

In [5]:
with open('./model/vectorizer.pickle', 'rb') as f:
    vectorizer = pickle.load(f)

In [6]:
tmp = []
tmp.append(test_lyric_seg)
test_lyric_vec = vectorizer.transform(tmp)

## Load Logistic Regression Model

In [7]:
LR_model = joblib.load('./model/LR_model')

## Predict category

In [8]:
predictions = LR_model.predict(test_lyric_vec)
predictions = predictions[0]
predictions

'勵志友情'

# Find similar lyric

In [9]:
if predictions == '愛情':
    category = 'love'
elif predictions == '勵志友情':
    category = 'friendship'
elif predictions == '悲傷離別':
    category = 'sad'
else:
    category = 'fight'

In [10]:
condition = df['Category'] == predictions
df_select = df[condition]
df_select = df_select.reset_index(drop = True)
df_select

,Singer,Name,Lyric,Category,seg_text,remove_synonym,top_10
0,A-Lin,One Life,白雲撞進藍天 激起昨天畫面我想起你的臉好久不見 在遠方的My Friend不論晴天雨天 也會...,勵志友情,"白雲撞, 進藍天, 激起, 昨天, 畫面, 想起, 臉, 好久, 不見, 遠方, MyFri...",白雲撞 進藍天 激起 昨天 畫面 想起 臉 好久 不見 遠處 MyFriend 晴天 雨天 ...,"['Life', '我們', '一天', '期待', '哪邊', '邊要', '出現', '..."
1,A-Lin,最好的朋友在身邊,一起笑一起哭一起鬧久別的回憶又浮現一點點一天天一年年最好的朋友在身邊一起走一起跑一起跳相聚到...,勵志友情,"一起, 笑, 一起, 哭, 一起, 鬧久別, 回憶, 浮現, 一點點, 一天天, 一年, 年...",一起 笑 一起 哭 一起 鬧久別 回憶 浮現 一些 一天天 一年 年 最好 朋友 身邊 一起...,"['一起', '一天天', '永在', '鬧久別', '回憶', '浮現', '一點點', ..."
2,A-Lin,愛的可能,你出現我身邊 像個奇蹟發生沒想到會是你 讓我如此失魂我心中的感覺是這樣陌生快樂的牽掛在相聚的...,勵志友情,"出現, 身邊, 蹟, 發生, 想到, 失魂, 心中, 感覺, 陌生, 快樂, 牽掛, 相聚,...",出現 身邊 蹟 發生 想 失魂 心中 感覺 陌生 快樂 牽掛 相見 每一分 曾以 見過 所有...,"['我會', '不會', '記得', '孤單', '時候', '因為', '還有', '我的..."
3,A-Lin,不是不滿足,朋友們 羨慕有他來接我回家不懂我是多盼望 能再多留一下她們說 自由很寬世界也很大哪裡海洋最浪...,勵志友情,"朋友, 羨慕, 接, 回家, 懂, 盼望, 再, 留, 一下, 說, 自由, 寬, 世界, ...",朋友 羨慕 接 回家 懂 盼 再 留 一下 說 自由 寬 世界 很大 海洋 最 浪漫 能治 ...,"['不滿足', '會想', '有過', '瘋狂', '糊塗', '膩了', '傾訴', '嫌..."
4,A-Lin,分手需要練習的,談起關於你的話題終於可以不用缺席甚至還表現得不再關心從前你是傷的痕跡現在不過是場回憶忽遠 忽...,勵志友情,"談起關, 話, 題終, 不用, 缺席, 還表現, 關心, 前, 是傷, 痕跡, 現在, 是場...",談起關 話 題終 不用 缺席 還表現 關心 前 是傷 痕跡 現在 是場 回憶 忽遠 忽近 低...,"['原來', '練習', '分手', '傷口', '不殘留', '一點', '回頭', '看..."
...,...,...,...,...,...,...,...
204,陳奕迅,愚人快樂,心總是總是晴朗 因為身邊有群好姑娘年少的人難免輕狂 於是遲暮年滿身的傷有時刻意地做一副愚蠢的...,勵志友情,"心總是, 總是, 晴朗, 身邊, 群好, 姑娘, 年少, 人, 難免, 輕狂, 遲, 暮年,...",心總是 總是 晴朗 身邊 群好 姑娘 年少 人 難免 輕狂 遲 暮年 滿身 傷有 時 刻意 ...,"['我們', '愛著', '因為', '親愛的', '陽光', '燦爛', '心總是', '..."
205,陳奕迅,新美人主義(國),不要一臉驚奇 看的 是你是你是你我們屏色靜氣 為的 是你是你是你你的智美兼具 活現 新的美人...,勵志友情,"一臉, 驚奇, 屏色, 靜氣, 智美, 兼具, 活現, 新, 美人主義, 多年, 一直, 尋...",一臉 驚奇 屏色 靜氣 智美 兼具 活現 新 美人主義 多年 一直 尋覓 懷疑 美麗本 這樣...,"['我們', '美麗', '思議', '有意思', '一臉', '驚奇', '屏色', '靜..."
206,陳奕迅,路一直都在,穿過人潮洶湧燈火闌珊沒有想過回頭一段又一段走不完的旅程甚麼時候能走完oh 我的夢代表甚麼又是...,勵志友情,"穿過, 人潮, 洶湧, 燈火, 闌, 珊, 想過, 回頭, 一段, 一段, 走, 完, 旅程...",穿越 人海 洶湧 燈 闌 珊 想過 回頭 一段 一段 走 完 旅行 走 完 oh 夢 代表 ...,"['甚麼', '時候', 'just', 'life', '一段', '穿過', '想過',..."
207,陳奕迅,暴風雨,擁有一切 在我手裡一場風雨 夜半來襲還以為人生全由人控制一覺醒來才發現一切隨陽光消失無法抗拒...,勵志友情,"擁有, 手裡, 一場, 風雨, 夜半, 來襲, 還以, 人生, 全由人, 控制, 一覺, 醒...",擁有 手裡 一場 風雨 夜半 來襲 還以 人生 全由人 控制 一覺 醒來 發現 隨陽光 消失...,"['暴風雨', '風雨', '事破壞', '遺再', '不過', '會過', '不可理喻',..."


In [11]:
# 載入同義字
word_net = []
with open("data/synonyms.txt", "r", encoding = "utf-8") as f1:
    for line in f1:
        word_net.append(line)

word_net = sorted(set(word_net))
synonyms_dic = {}

for word in word_net:
    word_s = word.split()
    synonyms_dic[word_s[0]] = word_s[1]

In [12]:
def synonyms(line):
    line_words = line.split()
    line_lyrics = ""
    for line_word in line_words:
        if line_word in synonyms_dic:
            line_lyrics = line_lyrics + synonyms_dic[line_word] + ' '
        else:
            line_lyrics = line_lyrics + line_word + ' '
    return line_lyrics

In [13]:
def getTopTen(line):
    words = jieba.analyse.extract_tags(line, 10)
    return words

In [14]:
doc = synonyms(test_lyric_seg)
doc_top10 = getTopTen(doc)

## Load LDA models

In [15]:
print("data/lyrics_{}.dict".format(category))

data/lyrics_friendship.dict


In [16]:
# 載入語料庫
if (os.path.exists("data/lyrics_{}.dict".format(category))):
    dictionary = corpora.Dictionary.load("data/lyrics_{}.dict".format(category)) # 建立Dictionary
    print(dictionary)
    corpus = corpora.MmCorpus("data/lyrics_{}.mm".format(category)) # 將數據流的語料變為內容流的語料
    print("Used files generated from first tutorial")
    print(corpus)
else:
    print("Please run first tutorial to generate data set")

Dictionary(6870 unique tokens: ['MyFriend', 'OneLife', '一些', '一天', '一起']...)
Used files generated from first tutorial
MmCorpus(209 documents, 6856 features, 11055 non-zero entries)


In [17]:
tfidf = models.TfidfModel(corpus)
# 轉為向量表示
corpus_tfidf = tfidf[corpus]

In [18]:
lda_model = models.LsiModel.load('model/lda_model_{}.lda'.format(category))
vec_bow = dictionary.doc2bow(doc.split()) # 把doc語料庫轉為一個一個詞包
vec_lsi = lda_model[vec_bow] # 用前面建好的 lsi 模型去計算這一篇歌詞 (input: 斷詞後的詞包、output: 20個主題成分)

In [19]:
index = similarities.MatrixSimilarity(lda_model[corpus])  # 建立索引
sims = index[vec_lsi] 
sims = sorted(enumerate(sims), key = lambda item: -item[1])

In [21]:
names, lyrics, top10 = [], [], []
for n, line, ten in zip(list(df_select['Name']), list(df_select['Lyric']), list(df_select['top_10'])):
    names.append(n)
    lyrics.append(line)
    top10.append(ten)

for lyric in sims[:5]:
    print("\n相似歌詞: ",  names[lyric[0]])
    print("TF-IDF top 10: ", top10[lyric[0]])
    print(lyrics[lyric[0]])
    print("相似度：",  lyric[1])


相似歌詞:   乾杯
TF-IDF top 10:  ['時間', '一天', '許會', '終點', '舉起', '回憶釀', '不會', '悠悠的', '真的', '一杯']
會不會 有一天 時間真的能倒退退回 你的我的 回不去的 悠悠的歲月也許會 有一天 世界真的有終點也要和你舉起回憶釀的甜 和你再乾一杯如果說 要我選出 代表青春 那個畫面浮現了 那滴眼淚 那片藍天 那年畢業那一張 邊哭邊笑 還要擁抱 是你的臉想起來 可愛可憐 可歌可泣 可是多懷念懷念總是 突然懷念 不談條件當回憶 衝破考卷 衝出歲月 在我眼前我和你 留著汗水 喝著汽水 在操場邊說好了 無論如何 一起走到 未來的世界現在就是 那個未來 那個世界為什麼 你的身邊 我的身邊 不是同一邊友情曾像 諾亞方舟 堅強誓言只是我 望著海面 等著永遠 模糊了視線會不會 有一天 時間真的能倒退退回 你的我的 回不去的 悠悠的歲月也許會 有一天 世界真的有終點也要和你舉起回憶釀的甜 和你再乾一杯這些年 買了四輪 買了手錶 買了單眼卻發現 追不到的 停不了的 還是那些人生是 只有認命 只能宿命 只好宿醉只剩下 高的笑點 低的哭點 卻沒成熟點成熟就是 幻想幻滅 一場磨鍊為什麼 只有夢想 越磨越小 小到不見有時候 好想流淚 好想流淚 卻沒眼淚期待會 你會不會 他會不會 開個同學會  他在等你 你在等我 我在等誰又是誰 孩子沒睡 電話沒電 心情沒準備天空不斷 黑了又亮 亮了又黑那光陰 滄海桑田 遠走高飛 再沒力氣追會不會 有一天 時間真的能倒退退回 你的我的 回不去的 悠悠的歲月也許會 有一天 世界真的有終點也要和你舉起回憶釀的甜 和你再乾一杯會不會 有一天 時間真的能倒退退回 你的我的 回不去的 悠悠的歲月也許會 有一天 世界真的有終點也要和你舉起回憶釀的甜 和你再乾一杯終究會 有一天 我們都變成昨天是你 陪我走過 一生一回 匆匆的人間有一天 就是今天 今天就是有一天說出一直沒說 對你的感謝 和你再乾一杯再乾一杯永遠 喝了就能萬歲 歲歲和年年 時間都停了 他們都回來了懷念的人阿 等你的來到
相似度： 0.98501146

相似歌詞:   最好的朋友在身邊
TF-IDF top 10:  ['一起', '一天天', '永在', '鬧久別', '回憶', '浮現', '一點點', '身邊', '永遠', '時光轉']
